<a href="https://colab.research.google.com/github/tutu-Jane/PhD-SpeechWPM-Analysis/blob/colab/%E5%85%85%E5%88%86%E5%88%A9%E7%94%A8%E6%82%A8%E7%9A%84_Colab_%E8%AE%A2%E9%98%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 充分利用您的 Colab 订阅



## 更多资源

### 在 Colab 中使用笔记本
- [Colab 概览](/notebooks/basic_features_overview.ipynb)
- [Markdown 指南](/notebooks/markdown_guide.ipynb)
- [导入库和安装依赖项](/notebooks/snippets/importing_libraries.ipynb)
- [在 GitHub 中保存和加载笔记本](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [互动表单](/notebooks/forms.ipynb)
- [互动微件](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### 处理数据
- [加载数据：云端硬盘、表格和 Google Cloud Storage](/notebooks/io.ipynb)
- [图表：可视化数据](/notebooks/charts.ipynb)
- [BigQuery 使用入门](/notebooks/bigquery.ipynb)

### 机器学习速成课程
以下是 Google 在线机器学习课程中的部分笔记本。请查看<a href="https://developers.google.com/machine-learning/crash-course/">完整课程网站</a>了解详情。
- [Pandas DataFrame 简介](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [通过 tf.keras 使用合成数据进行线性回归](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### 使用加速硬件
- [将 TensorFlow 与 GPU 配合使用](/notebooks/gpu.ipynb)
- [Colab 中的 TPU](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## 机器学习示例

如需查看 Colab 能够实现的互动机器学习分析的端到端示例，请参阅这些使用 <a href="https://tfhub.dev">TensorFlow Hub</a> 中模型的教程。

一些精选示例：

- <a href="https://tensorflow.org/hub/tutorials/tf2_image_retraining">重新训练图像分类器</a>：基于预训练的图像分类器，构建一个分辨花朵的 Keras 模型。
- <a href="https://tensorflow.org/hub/tutorials/tf2_text_classification">文本分类</a>：将 IMDB 影评分类为“褒义”<em></em>或<em></em>“贬义”。
- <a href="https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization">风格迁移</a>：使用深度学习在图像之间迁移风格。
- <a href="https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa">Multilingual Universal Sentence Encoder Q&amp;A</a>：使用机器学习模型回答来自 SQuAD 数据集中的问题。
- <a href="https://tensorflow.org/hub/tutorials/tweening_conv3d">视频插帧</a>：预测视频第一帧和最后一帧之间有些什么。


In [12]:
import os
import csv
import whisper
import pandas as pd
from tqdm import tqdm

# 设置路径
audio_folder = "/content/drive/MyDrive/音频统一"
output_csv = "/content/drive/MyDrive/wpm_result_501_cpu.csv"

# 加载 whisper 模型（建议使用 base 以避免内存问题）
model = whisper.load_model("base")

# 已完成列表（从CSV中读取）
completed_files = set()
if os.path.exists(output_csv):
    df_old = pd.read_csv(output_csv)
    completed_files = set(df_old["filename"])

# 增量转写追加到原文件
write_header = not os.path.exists(output_csv)
out_f = open(output_csv, "a", newline='', encoding="utf-8-sig")
writer = csv.DictWriter(out_f, fieldnames=["filename", "duration_sec", "text", "words", "wpm"])
if write_header:
    writer.writeheader()

# 遍历未完成的音频文件
for filename in tqdm(sorted(os.listdir(audio_folder))):
    if not filename.endswith(".wav") or filename in completed_files:
        continue

    filepath = os.path.join(audio_folder, filename)
    try:
        result = model.transcribe(filepath)
        text = result["text"].strip()
        duration = result["segments"][-1]["end"] if result["segments"] else 0
        word_count = len(text.split())
        wpm = round(word_count / (duration / 60), 2) if duration > 0 else 0

        writer.writerow({
            "filename": filename,
            "duration_sec": round(duration, 2),
            "text": text,
            "words": word_count,
            "wpm": wpm
        })
        print(f"✅ {filename} 完成")

    except Exception as e:
        print(f"❌ 处理失败: {filename}, 错误: {str(e)}")
        continue

out_f.close()
print("🎉 所有未完成音频转写已处理完毕")


  0%|          | 0/501 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
  0%|          | 1/501 [00:39<5:25:39, 39.08s/it]

✅ 520英语end.wav 完成


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
  0%|          | 2/501 [02:37<11:51:34, 85.56s/it]

✅ 520英语middle.wav 完成


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
  1%|          | 3/501 [03:04<8:11:19, 59.20s/it] 

...

✅ Amanda-middle.wav 完成


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
  1%|          | 6/501 [05:11<6:43:50, 48.95s/it]

✅ Amanda-start.wav 完成


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
  1%|          | 6/501 [05:39<7:46:12, 56.51s/it]


KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
import pandas as pd
import whisper

# 📍路径设置
audio_dir = '/content/drive/MyDrive/音频统一'
csv_path = '/content/drive/MyDrive/wpm_result_501_cpu.csv'

# ✅ 初始化：加载已处理文件名
existing = set()
if os.path.exists(csv_path):
    try:
        df_existing = pd.read_csv(csv_path)
        existing = set(df_existing['filename'].tolist())
        print(f"🔁 已找到 {len(existing)} 个历史处理记录，将自动跳过")
    except:
        print("⚠️ 警告：旧CSV文件无法读取，可能是空文件或损坏，将重新开始")

# ✅ 加载 Whisper 模型（建议 base 级别）
model = whisper.load_model("base")

# ✅ 处理音频
for filename in sorted(os.listdir(audio_dir)):
    if not filename.endswith('.wav'):
        continue
    if filename in existing:
        print(f"⏩ 已处理，跳过: {filename}")
        continue

    file_path = os.path.join(audio_dir, filename)
    try:
        result = model.transcribe(file_path)
        text = result['text']
        duration = result['segments'][-1]['end'] if result['segments'] else 0
        word_count = len(text.strip().split())
        wpm = round(word_count / (duration / 60), 2) if duration > 0 else 0

        row = pd.DataFrame([{
            'filename': filename,
            'duration_sec': round(duration, 2),
            'text': text,
            'words': word_count,
            'wpm': wpm
        }])

        # ✅ 追加写入 CSV（不覆盖）
        row.to_csv(csv_path, mode='a', index=False, header=not os.path.exists(csv_path) or os.stat(csv_path).st_size == 0, encoding='utf-8-sig')
        print(f"✅ {filename} 完成，已写入结果")

    except Exception as e:
        print(f"❌ 错误：{filename} 跳过，原因：{str(e)}")
        continue

print("🎉 所有音频处理完成！")


⚠️ 警告：旧CSV文件无法读取，可能是空文件或损坏，将重新开始


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ 520英语end.wav 完成，已写入结果


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ 520英语middle.wav 完成，已写入结果


...

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Mia英语end.wav 完成，已写入结果


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Mia英语middle.wav 完成，已写入结果


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
